<a href="https://colab.research.google.com/github/LianaN/TextLab/blob/main/FewShortLerning_with_LangChain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#! pip install openai
#! pip install langchain
#! pip install yt_dlp
#! pip install pydub
#!pip install git+https://github.com/huggingface/transformers sentencepiece datasets
#!pip install --upgrade azure-ai-formrecognizer > /dev/null
#!pip install --upgrade azure-cognitiveservices-speech > /dev/null
#!pip install --upgrade azure-ai-vision > /dev/null

In [2]:
import os

import openai

from langchain.llms import OpenAI
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.prompt import PromptTemplate
from langchain.chains import LLMChain
from langchain.agents import initialize_agent, AgentType

from langchain.agents.agent_toolkits import AzureCognitiveServicesToolkit
from langchain.tools.azure_cognitive_services.text2speech import AzureCogsText2SpeechTool

from gtts import gTTS
from pydub import AudioSegment

In [3]:
os.environ["OPENAI_API_KEY"] = "..."
os.environ["AZURE_COGS_KEY"] = "..."
os.environ["AZURE_COGS_ENDPOINT"] = "..."
os.environ["AZURE_COGS_REGION"] = "..."

In [4]:
prank_examples = [
  {
    "request":
"""
Escribe un guión para una broma telefónica.

Aquí están los ajustes de la broma:
Tema: Llamada desde la policía sobre SEAT Ibiza azul
Nombre del bromista: Sargento García
Nombre de la persona a la que se le hace la broma: Juan
""",
    "prank":
"""
Bromista: Hola, puedo hablar con el señor Juan?
Víctima: Sí, soy Juan. ¿Quién llama?
Bromista: Buenas tardes, señor Juan. Soy el Sargento García de la estación de policía local.
Víctima: Oh, ¿ocurre algo?
Bromista: Señor, ha habido un pequeño incidente que involucra un gato callejero y su vehículo. ¿Usted tiene un SEAT Ibiza azul?
Víctima: Sí, es correcto. ¿Qué pasó?
Bromista: A ver, este no es un gato común. Se trata de una raza especial, extremadamente rara y protegida, un Persa Azul. Parece que ha adoptado su coche como su territorio y ahora se niega a abandonarlo.
Víctima: ¿Qué? ¿Un gato... en mi coche?
Bromista: Está encima de su coche. Y, desafortunadamente, debido a su condición de raza protegida, legalmente no podemos retirarlo sin su permiso.
Víctima: Entonces, ¿qué quiere que haga al respecto?
Bromista: Bueno, señor, necesitamos que venga a la estación de policía local para firmar un formulario de liberación para poder reubicar al gato de manera segura.
Víctima: Me parece una locura.
Bromista: Es una situación bastante inusual. Gracias por comprender y cooperar.
"""
  },
  {
    "request":
"""
Escribe un guión para una broma telefónica.

Aquí están los ajustes de la broma:
Tema: Alquiler de barcos en el puerto de Mahón
Nombre del bromista: Sergio de la empresa Merak
Nombre de la persona a la que se le hace la broma: Ingrid
""",
    "prank":
"""
Bromista: ¡Hola, habla Sergio de la empresa de alquiler de barcos Merak en el puerto de Mahón! ¿Estoy hablando con Ingrid?
Víctima: Sí, soy yo.
Bromista: ¡Hola, Ingrid! Quería confirmar su reserva para nuestro yate de lujo "El Dorado" para el próximo fin de semana. Tenemos todo listo, solo necesitamos su confirmación final.
Víctima: ¿Qué? Yo no hice ninguna reserva...
Bromista: ¿En serio? Tenemos aquí una reserva a su nombre, incluyendo un servicio completo de catering, una banda en vivo y un espectáculo de fuegos artificiales. El total de la factura es de 50,000 euros.
Víctima: No, no, es imposible.
Bromista: Como que no? Usted ha hecho la reserva hace 7 días. La reserva incluye todo: el alquiler del yate, la comida, la música, el seguro del barco.
Víctima: No, que raro! Yo no hice la reserva...
Bromista: ¿En serio? Vaya, esto es un problema. Nuestra política de cancelación es de 72 horas. Pasado ese tiempo, los gastos ya están comprometidos y no podemos hacer devoluciones...
Víctima: Pero es un error.
Bromista: Mmmm... parece que hay un gran malentendido aquí, Ingrid. ¿Podría ser que alguien más hizo la reserva en tu nombre?
Víctima: No lo se...
Bromista: Sabes, esto suena muy extraño, pero... ¿Podría ser que tengas un admirador secreto que ha organizado una gran sorpresa para ti?
Víctima: Podría ser, pero me temo que es un error.
Bromista: Bueno, deberías prepararte para una gran fiesta en el yate, entonces. Te esperamos mañana a las 9:00 en nuestra oficina del puerto de Mahon.
"""
  },
]

In [5]:
prank_prompt = PromptTemplate(input_variables=["request", "prank"], template="Request: {request}\n{prank}")
print(prank_prompt.format(**prank_examples[0]))

Request: 
Escribe un guión para una broma telefónica.

Aquí están los ajustes de la broma:
Tema: Llamada desde la policía sobre SEAT Ibiza azul
Nombre del bromista: Sargento García
Nombre de la persona a la que se le hace la broma: Juan


Bromista: Hola, puedo hablar con el señor Juan?
Víctima: Sí, soy Juan. ¿Quién llama?
Bromista: Buenas tardes, señor Juan. Soy el Sargento García de la estación de policía local.
Víctima: Oh, ¿ocurre algo?
Bromista: Señor, ha habido un pequeño incidente que involucra un gato callejero y su vehículo. ¿Usted tiene un SEAT Ibiza azul?
Víctima: Sí, es correcto. ¿Qué pasó?
Bromista: A ver, este no es un gato común. Se trata de una raza especial, extremadamente rara y protegida, un Persa Azul. Parece que ha adoptado su coche como su territorio y ahora se niega a abandonarlo.
Víctima: ¿Qué? ¿Un gato... en mi coche?
Bromista: Está encima de su coche. Y, desafortunadamente, debido a su condición de raza protegida, legalmente no podemos retirarlo sin su permiso.

In [6]:
prompt = FewShotPromptTemplate(
    examples=prank_examples,
    example_prompt=prank_prompt,
    suffix="Request: {request}",
    input_variables=["request"]
)

In [7]:
llm = OpenAI(temperature=0, max_tokens=2000)

In [8]:
chain = LLMChain(llm=llm, prompt=prompt)
dialogue = chain.run(
"""
Escribe un guión para una broma telefónica.

Aquí están los ajustes de la broma:
Tema: Llamada desde el restaurante Paput Menorca sobre la reserva para esta tarde
Nombre del bromista: Francesc
Nombre de la persona a la que se le hace la broma: Judith
""")
print(dialogue)


Bromista: ¡Hola, habla Francesc del restaurante Paput Menorca! ¿Estoy hablando con Judith?
Víctima: Sí, soy yo.
Bromista: ¡Hola, Judith! Quería confirmar su reserva para esta tarde. Tenemos todo listo, solo necesitamos su confirmación final.
Víctima: ¿Qué? Yo no hice ninguna reserva...
Bromista: ¿En serio? Tenemos aquí una reserva a su nombre, para una mesa para dos personas. El menú incluye una selección de entrantes, un plato principal y un postre. El total de la factura es de 80 euros.
Víctima: No, no, es imposible.
Bromista: Como que no? Usted hizo la reserva hace dos horas. La reserva incluye todo: la comida, la bebida, el servicio.
Víctima: No, que raro! Yo no hice la reserva...
Bromista: ¿En serio? Vaya, esto es un problema. Nuestra política de cancelación es de 24 horas. Pasado ese tiempo, los gastos ya están comprometidos y no podemos hacer devoluciones...
Víctima: Pero es un error.
Bromista: Mmmm... parece que hay un gran malentendido aquí, Judith. ¿Podría ser que alguien má

In [9]:
dialogue = chain.run(
"""
Escribe un guión para una broma telefónica.

Aquí están los ajustes de la broma:
Tema: Llamada de una desconocida sobre la recogida de mejillones en la cala Padera
Nombre del bromista: Sofia
Nombre de la persona a la que se le hace la broma: Lidia
""")
print(dialogue)


Bromista: ¡Hola, habla Sofia! ¿Estoy hablando con Lidia?
Víctima: Sí, soy yo.
Bromista: ¡Hola, Lidia! Estoy llamando porque necesito tu ayuda. Estoy organizando una recogida de mejillones en la cala Padera y necesito que me ayudes.
Víctima: ¿Una recogida de mejillones? ¿Qué quieres que haga?
Bromista: Necesito que vengas a la cala Padera mañana a las 8:00 de la mañana. Trae una cesta grande y una pala.
Víctima: ¿Una pala? ¿Para qué?
Bromista: Para recoger los mejillones. Estamos organizando una recogida masiva para una fiesta de cumpleaños.
Víctima: ¿Una fiesta de cumpleaños? ¿Quién es el afortunado?
Bromista: Bueno, es una sorpresa. Pero necesitamos tu ayuda para recoger los mejillones. ¿Puedes venir mañana a las 8:00?
Víctima: No estoy segura de que sea una buena idea...
Bromista: Por favor, Lidia. Será divertido. Y, además, te prometo que te llevaré a la fiesta de cumpleaños para que descubras quién es el afortunado.
Víctima: Bueno, supongo que no hay problema. Está bien, estaré al

In [10]:
dialogue = chain.run(
"""
Escribe un guión para una broma telefónica.

Aquí están los ajustes de la broma:
Tema: Llamada de escuela online ILERNA. Ingrid no podrá realizar las prácticas de radiografía este año y tendrá que esperar hasta el año que viene.
Nombre del bromista: Carlos de ILERNA
Nombre de la persona a la que se le hace la broma: Ingrid
""")
print(dialogue)


Bromista: Hola, ¿puedo hablar con Ingrid?
Víctima: Sí, soy yo.
Bromista: Hola, Ingrid. Soy Carlos de ILERNA, la escuela online de radiografía.
Víctima: ¿Qué pasa?
Bromista: Bueno, hemos estado revisando tu expediente y lamentablemente no podrás realizar las prácticas de radiografía este año.
Víctima: ¿Qué? ¿Por qué?
Bromista: Bueno, parece que hay un problema con tu expediente. Parece que no has cumplido con los requisitos necesarios para realizar las prácticas.
Víctima: ¿Qué requisitos? Yo pensé que había cumplido con todos los requisitos.
Bromista: Bueno, hay un requisito que no has cumplido. Parece que no has completado el curso de "Cocina para Radiólogos".
Víctima: ¿Qué? ¿Cocina para Radiólogos? ¿Qué tiene que ver eso con radiografía?
Bromista: Bueno, es un requisito obligatorio para todos los estudiantes de radiografía. Debes completar el curso para poder realizar las prácticas.
Víctima: Pero yo no sabía nada de eso.
Bromista: Lo siento, Ingrid. Tendrás que esperar hasta el año q

In [16]:
from transformers import SpeechT5Processor, SpeechT5ForTextToSpeech, SpeechT5HifiGan
from datasets import load_dataset
import torch
import soundfile as sf
from datasets import load_dataset

processor = SpeechT5Processor.from_pretrained("microsoft/speecht5_tts")
model = SpeechT5ForTextToSpeech.from_pretrained("microsoft/speecht5_tts")
vocoder = SpeechT5HifiGan.from_pretrained("microsoft/speecht5_hifigan")

inputs = processor(text="Hola Ingrid! Soy Carlos de ILEARNA", return_tensors="pt")

# load xvector containing speaker's voice characteristics from a dataset
embeddings_dataset = load_dataset("Matthijs/cmu-arctic-xvectors", split="validation")
speaker_embeddings = torch.tensor(embeddings_dataset[7306]["xvector"]).unsqueeze(0)

speech = model.generate_speech(inputs["input_ids"], speaker_embeddings, vocoder=vocoder)

sf.write("speech.wav", speech.numpy(), samplerate=16000)

In [17]:
from IPython.display import Audio

Audio(speech.numpy(), rate=16000)